In [1]:
from sklearn import datasets, model_selection, linear_model, metrics, preprocessing
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
% matplotlib inline

In [2]:
def train_model(clf_name, data, labels, random_state=10, cv=4, normalize=False):
    cv = model_selection.ShuffleSplit(n_splits=cv,random_state=random_state)
    accs = []
    rcls = []
    prcs = []
    for train_inds, test_inds in cv.split(data):
        if clf_name == 'log':
            clf = linear_model.SGDClassifier(loss='log', random_state=random_state)    
        train_data = data[train_inds]
        train_labels = labels[train_inds]
        test_data = data[test_inds]
        test_labels = labels[test_inds]
        if normalize:
            normalizer = preprocessing.Normalizer().fit(train_data)
            train_data = normalizer.transform(train_data)
            test_data = normalizer.transform(test_data)
        clf.fit(train_data, train_labels)
        accs.append(clf.score(test_data, test_labels))
        rcls.append(metrics.recall_score(y_true=test_labels, y_pred=clf.predict(test_data), pos_label=0))
        prcs.append(metrics.precision_score(y_true=test_labels, y_pred=clf.predict(test_data), pos_label=0))  
    mean_acc = np.array(accs).mean()
    mean_prc = np.array(prcs).mean()
    mean_rcl = np.array(rcls).mean()  
    return mean_acc, mean_rcl, mean_prc  

In [3]:
bc = datasets.load_breast_cancer()
acc, prc, rcl = train_model('log', bc.data, bc.target, normalize=True)
print 'Accuracy:', acc
print 'Precision:', prc
print 'Recall:', rcl

Accuracy: 0.8771929824561404
Precision: 0.7784679089026916
Recall: 0.9200396825396826


D:\Anaconda\envs\py27\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [4]:
imb_bc = pd.read_csv('./imbalanced_datasets/imbalanced_breast_cancer.csv')
imb_bc_labels = np.array(imb_bc.label)
imb_bc_data = np.array(imb_bc.drop('label', axis=1))        
        
acc, prc, rcl = train_model('log', imb_bc_data, imb_bc_labels, normalize=True)
print 'Accuracy:', acc
print 'Precision:', prc
print 'Recall:', rcl

Accuracy: 0.9605263157894737
Precision: 0.125
Recall: 0.25


D:\Anaconda\envs\py27\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
